## Imports

Make sure to import all of the necessary dependencies for your model to operate. Both pipeline 1 and 2 should include the following:

1. utils.py
2. Path to the child model class
3. Any extra dependencies unrelated to the class

In [1]:
from utils import *
# (TODO) Rest of the imports (Anyone)
from BiLSTM_Module2 import BiLSTM_Module2
from LSTM.LSTM import LSTM

## Instantiating the Modules

First, store the path to the test dataset

In [2]:
# (TODO) Adjust the path to the test set if needed (Anyone)
# Note: This is the ORIGINAL test data, the one that contains test.SRC, test.TOP, etc..
# If you don't have it, download it from the repo at https://github.com/amazon-science/pizza-semantic-parsing-dataset/tree/main/data
test_data_dir = "dataset/PIZZA_test.json"

Then, load modules 1 and 2 using the classes you created. You can change the constructor to take any parameters; just make sure that you classes contain an implementation of the "predict_labels" method and takes no extra parameters.

In [3]:
# (TODO) Instantiate module 1 (Hany and Marwan)
model_path = "LSTM\lstm.keras"
word2vec_path = "word2vec-google-news-300"
module1 = LSTM(model_path, word2vec_path)

<>:2: SyntaxWarning: invalid escape sequence '\l'
<>:2: SyntaxWarning: invalid escape sequence '\l'
C:\Users\hassa\AppData\Local\Temp\ipykernel_26244\2476852733.py:2: SyntaxWarning: invalid escape sequence '\l'
  model_path = "LSTM\lstm.keras"


Loading word2vec model from disk...


In [7]:
# (TODO) Instantiate module 2 (Tyson)
model_path="sequence_labelling_final_model_2.keras"
module2 = BiLSTM_Module2(model_path,word2vec_path)

Loading word2vec model from disk... module2


Done! The rest of the code should not be changed. Just run all :)

In [8]:
sentence = "i would like to try one large chicken and mushroom pizza with no extra cheese please"

print("Input Order: " + sentence)

# Segment orders
isa_words, isa_labels = module1.predict_labels(sentence)
orders = segment_orders(isa_words, isa_labels)
top_decoupled = ''
for order_pair in orders:
    order = order_pair[0]
    is_pizza = order_pair[1]
    parser_words, parser_labels = module2.predict_labels(order)
    if is_pizza: id = 0
    else: id = 1

    top_decoupled += (generate_top_decoupled(order, parser_labels, id) + ' ')

print("Top Decoupled: " + top_decoupled[:-1])

Input Order: i would like to try one large chicken and mushroom pizza with no extra cheese please
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 625ms/step
Top Decoupled: (PIZZAORDER (NUMBER one ) (SIZE large ) (TOPPING chicken ) (TOPPING mushroom ) (NOT (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) ) )


In [9]:
import json
correct_count = 0
i = 0
with open(test_data_dir, 'r') as infile:
    for line in infile:
        instance = json.loads(line)
        input_sentence = instance.get(f"test.SRC", "")
        top = instance.get(f"test.TOP", "")
        true_top_decoupled = generate_top_decoupled_from_top(top)

        isa_words, isa_labels = module1.predict_labels(input_sentence)
        orders = segment_orders(isa_words, isa_labels)
        top_decoupled = ''
        for order_pair in orders:
            order = order_pair[0]
            is_pizza = order_pair[1]
            parser_words, parser_labels = module2.predict_labels(order)

            if is_pizza: id = 0
            else: id = 1

            top_decoupled += (generate_top_decoupled(order, parser_labels, id) + ' ')
        
        top_decoupled = "(ORDER " + top_decoupled + ')'

        if top_decoupled == true_top_decoupled: correct_count += 1
        else:
            print(f"Row Index: {i}")
            print("Input Sentence: " + input_sentence)
            print("True Output: " + true_top_decoupled)
            print("Predicted Output: " + top_decoupled)
            print("----------------------------------------------------------------------")
        

        i += 1

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 586ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━

In [10]:
correct_count / i

0.9727339719970524